In [1]:
# dependencies
import pandas as pd

In [2]:
# support methods
def open_gz(f):
    return pd.read_csv(f, compression='gzip')

In [3]:
# __main__
news_included = '../input/news_articles_matchedsentence.csv.gz'
news_true = '../input/news_articles_matchedsentence_officers.csv.gz'
news_text = '../input/news_articles_newsarticle.csv.gz'

inc_df = open_gz(news_included)
true_df = open_gz(news_true)
text_df = open_gz(news_text)

In [4]:
inc_df

,id,created_at,updated_at,article_id,extracted_keywords,text,title
0,14007,2022-04-19T07:05:54.822Z,2022-04-19T07:05:55.517Z,31382,"[""Police""]",The Denham Springs Police Department is stress...,"Around Livingston for April 20, 2022"
1,14006,2022-04-19T07:05:53.589Z,2022-04-19T07:05:54.670Z,31379,"[""officer""]","She met with Scot Byrd, chief administrative o...","Garden Club meets, tours grounds of historic H..."
2,13997,2022-04-19T07:04:12.149Z,2022-04-19T07:04:14.578Z,31343,"[""Officer""]","Corrections Officer Elbert King, 30, of Natche...",Angola guard booked for excessive force; left ...
3,13996,2022-04-19T07:04:10.112Z,2022-04-19T07:04:11.309Z,31331,"[""police""]",Devin Jr.’s death appeared to be the year’s fi...,Family of slain 3-year-old begs for end to Bat...
4,13995,2022-04-19T07:04:10.090Z,2022-04-19T07:04:10.107Z,31331,"[""Police""]","Still, guns wreaked havoc through the weekend,...",Family of slain 3-year-old begs for end to Bat...
...,...,...,...,...,...,...,...
10465,5,2021-10-19T07:00:53.150Z,2022-03-05T07:08:56.799Z,1643,"[""police""]","If anyone sees suspicious activities, call the...","Around the Felicianas for Oct. 20, 2021"
10466,4,2021-10-19T07:00:52.419Z,2022-03-05T07:07:47.462Z,709,"[""Officer""]",The Advocate reported LDH State Health Officer...,New Orleans school district comes out against ...
10467,3,2021-10-19T07:00:51.123Z,2022-03-05T07:08:56.809Z,366,"[""Officer""]",At a Monday press conference with New Orlean...,"As initial FEMA deadline approaches, some appl..."
10468,2,2021-10-19T07:00:50.862Z,2022-03-05T07:08:19.624Z,365,"[""Officer""]","Before students return to any campus, NOLA P...",Some New Orleans schools will start back virtu...


In [5]:
true_df

,id,matchedsentence_id,officer_id
0,61486,13992,61486
1,115086,13992,115086
2,110173,13632,110173
3,138540,13632,138540
4,101010,13632,101010
...,...,...,...
730,73003,88,73003
731,76055,82,76055
732,73862,81,73862
733,72518,76,72518


In [6]:
text_df

,created_at,link,guid,source_id,updated_at,content,published_date,id,title,is_processed,author,url
0,2022-04-19T07:02:31.558Z,http://klax-tv.com/how-to-cope-with-diabetes/?...,78128,4,2022-04-19T07:05:55.559Z,Living with diabetes is a lifetime challenge f...,2022-04-14,31383,How to Cope with Diabetes,True,Keisha Swafford,https://storage.googleapis.com/llead-documents...
1,2022-04-19T07:01:17.106Z,https://www.theadvocate.com/baton_rouge/news/c...,ee7ff630-ba85-11ec-bbb7-c3fe87dfb42d,2,2022-04-19T07:05:55.521Z,Spring Fest is Saturday The Denham Springs Spr...,2022-04-19,31382,"Around Livingston for April 20, 2022",True,Leila Pitchford,https://storage.googleapis.com/llead-documents...
2,2022-04-19T07:01:16.876Z,https://www.theadvocate.com/baton_rouge/news/c...,8d45c8ac-be01-11ec-abd9-274b18372031,2,2022-04-19T07:04:38.003Z,As the Live Oak softball team begins postseaso...,2022-04-19,31381,Live Oak softball enters playoffs 'being able ...,True,Charles Salzer,https://storage.googleapis.com/llead-documents...
3,2022-04-19T07:01:16.643Z,https://www.theadvocate.com/baton_rouge/news/c...,2c5af6f6-ba7e-11ec-8d1f-cfd7e62045b6,2,2022-04-19T07:05:54.758Z,North Oaks Sports Medicine certified athletic ...,2022-04-19,31380,North Oaks Sports Medicine selects student-ath...,True,Community News Report,https://storage.googleapis.com/llead-documents...
4,2022-04-19T07:01:16.418Z,https://www.theadvocate.com/baton_rouge/news/c...,22fc3b76-be5f-11ec-8684-37c558587c31,2,2022-04-19T07:05:54.675Z,The Gonzales Garden Club gathered for a busine...,2022-04-19,31379,"Garden Club meets, tours grounds of historic H...",True,Community News Report,https://storage.googleapis.com/llead-documents...
...,...,...,...,...,...,...,...,...,...,...,...,...
29702,2021-09-29T06:00:34.440Z,https://thelensnola.org/2021/09/05/a-week-afte...,566903,1,2021-10-19T07:00:54.226Z,One week after Hurricane Ida made landfall in ...,2021-09-05,356,"A week after Hurricane Ida, 61 percent of cust...",True,Michael Isaac Stein,https://storage.googleapis.com/llead-documents...
29703,2021-09-29T06:00:34.156Z,https://thelensnola.org/2021/09/06/eight-days-...,566943,1,2021-10-19T07:00:49.195Z,Eight days after Hurricane Ida barreled throug...,2021-09-06,355,"Eight days after Hurricane Ida, 71 percent of ...",True,Michael Isaac Stein,https://storage.googleapis.com/llead-documents...
29704,2021-09-29T06:00:33.883Z,https://thelensnola.org/2021/09/07/nine-days-a...,567000,1,2021-10-19T07:05:26.309Z,Entergy executives announced moderate overnigh...,2021-09-07,354,"Nine days after Hurricane Ida, 73 percent of N...",True,Michael Isaac Stein,https://storage.googleapis.com/llead-documents...
29705,2021-09-29T06:00:33.605Z,https://thelensnola.org/2021/09/07/governor-ed...,566989,1,2021-10-19T07:00:52.223Z,Gov. John Bel Edwards has issued an executive ...,2021-09-07,353,Governor Edwards suspends evictions until Sept...,True,Philip Kiefer,https://storage.googleapis.com/llead-documents...
